In [2]:
import numpy as np
import pandas as pd
import regex as re
from langdetect import detect
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from PIL import Image
from textblob import TextBlob
from wordcloud import STOPWORDS, WordCloud

<h3>Read previously saved CSV

In [1]:
#filename = "tweets_Lululemon_langs_translated.csv"
filename = "Lululemon_tweets_en.csv"
df = pd.read_csv(filename, index_col='id', low_memory = False)

<IPython.core.display.Javascript object>

In [2]:
#convert datatypes
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = pd.to_datetime(df['date'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
df.head()

,timestamp,content,username,date,clean_content,lang,translated_content
id,,,,,,,
1174824778609225729,2019-09-19 23:17:09+00:00,Calling my name again....@lululemon https://t....,keepitrealmom,2019-09-19,Calling my name againlululemon,tl,Calling my name againlululemon
1174810733210742785,2019-09-19 22:21:20+00:00,PSA: llego la colección exclusiva de Robert Ge...,lululemonMX,2019-09-19,PSA llego la coleccin exclusiva de Robert Gell...,es,PSA arrived the exclusive collection of Robert...
1174807747558117376,2019-09-19 22:09:28+00:00,@YogaKittyHawks lululemonは知らない人にとっては、ただの運動着(死語...,70TerraVistaSF,2019-09-19,lululemon,ro,lululemon
1174796278259208192,2019-09-19 21:23:54+00:00,lululemon Resources and Topics https://t.co/Po...,trendycyndie,2019-09-19,lululemon Resources and Topics via,ro,lululemon Resources and Topics via
1174791699329146880,2019-09-19 21:05:42+00:00,@henrykleeKTVU @KTVU @lululemon @CHPSantaRosa ...,100ForestSt,2019-09-19,Stole Rob maybe,tl,Stole Rob maybe


In [6]:
work_df_clean = df.drop_duplicates(subset=['clean_content'])
work_df_clean['neg'] = 0
work_df_clean['neu'] = 0
work_df_clean['pos'] = 0
work_df_clean['compound'] = 0
work_df_clean['sentiment'] = ''
work_df_clean[['polarity', 'subjectivity']] = work_df_clean['clean_content'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

/var/folders/16/0qktjm550_74tvv2gs54_lzc0000gn/T/ipykernel_99029/2493134778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  work_df_clean['neg'] = 0
/var/folders/16/0qktjm550_74tvv2gs54_lzc0000gn/T/ipykernel_99029/2493134778.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  work_df_clean['neu'] = 0
/var/folders/16/0qktjm550_74tvv2gs54_lzc0000gn/T/ipykernel_99029/2493134778.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [8]:
#Calculating Negative, Positive, Neutral and Compound values
def calc_sent (row_local):
    score = SentimentIntensityAnalyzer().polarity_scores(row_local['clean_content'])
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    row_local['neg'] = neg
    row_local['neu'] = neu
    row_local['pos'] = pos
    row_local['compound'] = comp
    if neg > pos:
        sentiment = "negative"
    elif pos > neg:
        sentiment = "positive"
    else:
        sentiment = "neutral"
    row_local['sentiment'] = sentiment
    return row_local
work_df_clean = work_df_clean.apply(calc_sent, axis='columns')
work_df_clean.head()

,timestamp,content,username,date,clean_content,lang,neg,neu,pos,compound,sentiment,polarity,subjectivity
id,,,,,,,,,,,,,
1042563491851390977,2018-09-19 23:58:01+00:00,Hey @lululemonCHI @lululemon I stopped at your...,jleboyer,2018-09-19,Hey I stopped at your running station in Chica...,en,0.105,0.788,0.108,0.2734,positive,0.5,0.625
1042563239786229760,2018-09-19 23:57:00+00:00,@ahmedtheelder You know what that means:https:...,lululemon,2018-09-19,You know what that meanshttpstcoiTRlSZik7d,en,0.000,1.000,0.000,0.0000,neutral,0.0,0.000
1042560954431623169,2018-09-19 23:47:56+00:00,@_danalouise MFM???? One of my favorite episod...,pjscaggs,2018-09-19,MFM One of my favorite episodes Lululemon is s...,en,0.000,0.750,0.250,0.4588,positive,0.5,1.000
1042559817691394048,2018-09-19 23:43:25+00:00,"PRESS RELEASE: Burglary and Grand Theft; ""Grab...",SantaRosaPolice,2018-09-19,PRESS RELEASE Burglary and Grand Theft Grab an...,en,0.000,0.812,0.188,0.4588,positive,0.5,1.000
1042558723854663680,2018-09-19 23:39:04+00:00,Ready for #SeaWheeze this weekend!\n•\n•\n•\n#...,shiftwitz,2018-09-19,Ready for SeaWheeze this weekend vancouver can...,en,0.000,0.828,0.172,0.3612,positive,0.1,0.250


In [9]:
work_df_clean.to_csv('Lulu_tweets_w_sentiment_en.csv')